<a href="https://colab.research.google.com/github/dalatorrem/Diplomado_Metodos_UCentral_2023/blob/main/Lectura_21_tests_intervalos_resultados_saber_11_mod_cla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

<p align="center">
  <img src="https://raw.githubusercontent.com/lacamposm/Diplomado_Metodos_UCentral/main/data/images/imagen_ucentral.jpg" alt="logo_Ucentral" width="400px" height="300px">
</p>

# **Análisis del impacto de variables sociodemográficas sobre  el resultado global de la prueba Saber 11**

*Objetivos*
Determinar si existen cambios significativos entre las distribuciones del resultado global de la prueba Saber 11 en grupos con variables sociodemográficas distintas.

Temas a revisar:

Estadística Desciptiva. Pruebas de hipótesis, intervalos de confianza, estimaciones puntuales, p-valor, nivel de significancia, nivel de confianza.


*Introducción* <br>
La prueba Saber 11 es presentada por los estudiantes de últimos grados del bachillerato como requisito para graduarse y por bachilleres que deseen volver a presentarla para mejorar su puntaje.

En muchas instituciones de educación superior un puntaje mínimo es requisito para el ingreso, en otras condiciona los cursos de primer semestre.

**Problema comercial:** *Determinar si se debe invertir recursos en determinados grupos para intentar dsiminuir las brechas en los resultados de la prueba y proponer intervalos para el promedio de puntajes globales en diferentes grupos o a nivel general*

**Contexto analítico:** El conjunto de datos proporcionado es una muestra aleatoria de resultados de la prueba saber 11.

**Se debe realizar:**

* Análisis exploratorio y limpieza de los datos.
* Análisis Descriptivo de los datos.
* Inferencia estadística.

In [1]:
%%capture
!pip install pingouin ## instalación libreria pingouin

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import t, nct   ## distribuciones t y t no-central.
from pingouin import ttest       ## ttest: prueba-t
import plotly.express as px
sns.set_style("darkgrid")

In [3]:
root_data = 'https://raw.githubusercontent.com/dalatorrem/Diplomado_Metodos_UCentral_2023/main/data/'

In [12]:
data_file = 'Resultados__nicos_Saber_11_pun_global_estrato_per_calendario_sample.csv'
df = pd.read_csv(f'{root_data}{data_file}')
df

,Unnamed: 0,PERIODO,COLE_CALENDARIO,FAMI_ESTRATOVIVIENDA,PUNT_GLOBAL
0,3843247,20194,A,Estrato 3,250.0
1,4559367,20162,A,Estrato 3,291.0
2,3001211,20194,A,Estrato 3,295.0
3,751042,20172,A,Estrato 1,238.0
4,5294012,20112,A,Estrato 1,NaN
...,...,...,...,...,...
49995,430149,20122,A,Estrato 3,NaN
49996,2770796,20112,A,Estrato 1,NaN
49997,4445065,20152,A,Estrato 2,225.0
49998,4016777,20102,A,Estrato 3,NaN


# Análisis Exploratorio

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            50000 non-null  int64  
 1   PERIODO               50000 non-null  int64  
 2   COLE_CALENDARIO       49989 non-null  object 
 3   FAMI_ESTRATOVIVIENDA  48780 non-null  object 
 4   PUNT_GLOBAL           28418 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 1.9+ MB


Qué hacer con los nulos?

Reemplazar por una palabra, o por un valor numérico
Imputar por el promedio
 Se deben eliminar los nulos en la varaible de interés

In [15]:
df = df.dropna(subset=['PUNT_GLOBAL']).drop(columns='Unnamed: 0')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28418 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PERIODO               28418 non-null  int64  
 1   COLE_CALENDARIO       28418 non-null  object 
 2   FAMI_ESTRATOVIVIENDA  27551 non-null  object 
 3   PUNT_GLOBAL           28418 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1.1+ MB


In [17]:
df['ANO'] = [int(str(x)[:-1]) for x in df['PERIODO']]
df

,PERIODO,COLE_CALENDARIO,FAMI_ESTRATOVIVIENDA,PUNT_GLOBAL,ANO
0,20194,A,Estrato 3,250.0,2019
1,20162,A,Estrato 3,291.0,2016
2,20194,A,Estrato 3,295.0,2019
3,20172,A,Estrato 1,238.0,2017
8,20152,A,Estrato 3,228.0,2015
...,...,...,...,...,...
49992,20194,A,Estrato 3,295.0,2019
49993,20194,A,Estrato 3,252.0,2019
49994,20162,A,Estrato 3,339.0,2016
49997,20152,A,Estrato 2,225.0,2015


# Descriptivo

In [18]:
df['FAMI_ESTRATOVIVIENDA'].value_counts().reset_index()

,index,FAMI_ESTRATOVIVIENDA
0,Estrato 1,10416
1,Estrato 2,9540
2,Estrato 3,5145
3,Estrato 4,1292
4,Estrato 5,469
5,Sin Estrato,390
6,Estrato 6,299


In [19]:
df['COLE_CALENDARIO'].value_counts().reset_index()

,index,COLE_CALENDARIO
0,A,27371
1,B,892
2,OTRO,155


In [20]:
df['ANO'].value_counts().reset_index()

,index,ANO
0,2019,9053
1,2015,4820
2,2016,4765
3,2017,4758
4,2014,4484
5,2018,256
6,2020,150
7,2021,132


In [23]:
df[['PUNT_GLOBAL']].describe()

,PUNT_GLOBAL
count,28418.000000
mean,252.731930
std,49.780866
min,0.000000
25%,216.000000
50%,248.000000
75%,286.000000
max,473.000000


Una cuarta parte de las personas que presentaron la prueba no obtienen  más de 216   puntos

La mitad de las personas que presentaron la prueba no obtienen  más de 248   puntos

Sólo una cuarta parte de las personas que presentaron la prueba  obtienen más de 286 puntos

In [25]:
(df['PUNT_GLOBAL']>=300).sum()/len(df)*100

17.868956295305793

## Tabla cruzada de tipo colegio y estrato

In [30]:
df_tabla_cro = round(100*pd.crosstab(df['COLE_CALENDARIO'], df['FAMI_ESTRATOVIVIENDA'], normalize='index'),1)
df_tabla_cro

FAMI_ESTRATOVIVIENDA,Estrato 1,Estrato 2,Estrato 3,Estrato 4,Estrato 5,Estrato 6,Sin Estrato
COLE_CALENDARIO,,,,,,,
A,39.1,35.3,18.5,4.1,1.1,0.5,1.4
B,2.9,12.6,21.1,23.9,19.2,19.8,0.6
OTRO,12.1,49.6,29.8,5.7,1.4,0.0,1.4


In [35]:
px.imshow(df_tabla_cro, text_auto = True)

**Reto: Realice un contraste de hipótesis para validar si las variables son independientes o no.**

## Plots

In [36]:
px.box(df['PUNT_GLOBAL'])

In [38]:
px.box(df.sort_values('FAMI_ESTRATOVIVIENDA'), x='PUNT_GLOBAL', y='FAMI_ESTRATOVIVIENDA', orientation='h')

In [39]:
px.violin(df.sort_values('FAMI_ESTRATOVIVIENDA'), x='PUNT_GLOBAL', y='FAMI_ESTRATOVIVIENDA', orientation='h')

In [40]:
px.violin(df.sort_values('FAMI_ESTRATOVIVIENDA'), x='PUNT_GLOBAL', orientation='h')

In [42]:
px.histogram(df['PUNT_GLOBAL'])

**Recuerden que los datos están desbalanceados en los periodos** *

In [43]:
px.box(df, y='PUNT_GLOBAL', x='ANO')

In [44]:
df.groupby(['FAMI_ESTRATOVIVIENDA']).agg({'PUNT_GLOBAL':['mean', np.median, 'std', 'count']})

PUNT_GLOBAL                         
                            mean median        std  count
FAMI_ESTRATOVIVIENDA                                     
Estrato 1             237.281970  233.0  41.866304  10416
Estrato 2             254.538155  252.0  45.463140   9540
Estrato 3             271.010107  271.0  48.632760   5145
Estrato 4             289.297988  295.0  60.836910   1292
Estrato 5             301.752665  311.0  63.914428    469
Estrato 6             311.956522  322.0  65.449515    299
Sin Estrato           215.551282  206.0  44.929893    390

# Inferencia estadística

## Revisar normalidad

Nivel de significancia del 0.01

In [45]:
import  scipy.stats as test_sc

Test Sobre el puntaje global (general)

$H_0$ Asimetría es la de una distribución normal. <br>
$H_A$ Asimetría NO es la de una distribución normal.

In [46]:
test_sc.skewtest(df['PUNT_GLOBAL'])

SkewtestResult(statistic=24.7667666760876, pvalue=2.045626340020716e-135)

Se rechaza la hipótesis nula de que la asimetria sea la de una normal (p_valor aproximadamente 0)

Test Sobre el puntaje global en cada uno de los estratos

$H_0$ Asimetría es la de una distribución normal. <br>
$H_A$ Asimetría NO es la de una distribución normal.

In [51]:
for est in df['FAMI_ESTRATOVIVIENDA'].unique():
  mask = df['FAMI_ESTRATOVIVIENDA']== est
  if mask.sum()>10:
    print('PVALOR de la prueba de asimetría en el puntaje global para el estrato:',est,
          round(test_sc.skewtest(df.loc[mask,'PUNT_GLOBAL']).pvalue,4))

PVALOR de la prueba de asimetría en el puntaje global para el estrato: Estrato 3 0.0996
PVALOR de la prueba de asimetría en el puntaje global para el estrato: Estrato 1 0.0
PVALOR de la prueba de asimetría en el puntaje global para el estrato: Estrato 2 0.0
PVALOR de la prueba de asimetría en el puntaje global para el estrato: Estrato 4 0.0025
PVALOR de la prueba de asimetría en el puntaje global para el estrato: Estrato 5 0.0007
PVALOR de la prueba de asimetría en el puntaje global para el estrato: Estrato 6 0.0
PVALOR de la prueba de asimetría en el puntaje global para el estrato: Sin Estrato 0.0


***En estrato 3 no rechazo la hipótesis nula de que la distribución tenga asimetría 0***

In [52]:
px.histogram(df.loc[(df['FAMI_ESTRATOVIVIENDA']=='Estrato 3'),'PUNT_GLOBAL'])

In [53]:
px.histogram(df.loc[(df['FAMI_ESTRATOVIVIENDA']=='Estrato 6'),'PUNT_GLOBAL'])

In [54]:
px.histogram(df.loc[(df['FAMI_ESTRATOVIVIENDA']=='Estrato 1'),'PUNT_GLOBAL'])

Test de normalidad para el puntaje global por estrato

$H_0$: La muestra proviene de una normal. <br>
$H_1$: La muestra NO proviene de una normal. <br>

In [55]:
for est in df['FAMI_ESTRATOVIVIENDA'].unique():
  mask = df['FAMI_ESTRATOVIVIENDA']== est
  if mask.sum()>10:
    print('PVALOR de la prueba de normalidad en el puntaje global para el estrato:',est,
          round(test_sc.normaltest(df.loc[mask,'PUNT_GLOBAL']).pvalue,4))

PVALOR de la prueba de normalidad en el puntaje global para el estrato: Estrato 3 0.0
PVALOR de la prueba de normalidad en el puntaje global para el estrato: Estrato 1 0.0
PVALOR de la prueba de normalidad en el puntaje global para el estrato: Estrato 2 0.0
PVALOR de la prueba de normalidad en el puntaje global para el estrato: Estrato 4 0.0
PVALOR de la prueba de normalidad en el puntaje global para el estrato: Estrato 5 0.0005
PVALOR de la prueba de normalidad en el puntaje global para el estrato: Estrato 6 0.0001
PVALOR de la prueba de normalidad en el puntaje global para el estrato: Sin Estrato 0.0


En todos los estratos se rechaza la hipótesis nula, por lo tanto no se puede asumir normalidad.

Test de normalidad para el puntaje global por estrato y calendario de colegio y periodo

$H_0$: La muestra proviene de una normal. <br>
$H_1$: La muestra NO proviene de una normal. <br>

In [56]:
df.columns

Index(['PERIODO', 'COLE_CALENDARIO', 'FAMI_ESTRATOVIVIENDA', 'PUNT_GLOBAL',
       'ANO'],
      dtype='object')

In [61]:
for est in df['FAMI_ESTRATOVIVIENDA'].unique():
  for per in df['PERIODO'].unique():
    for cal in df['COLE_CALENDARIO'].unique():
      mask = (df['FAMI_ESTRATOVIVIENDA']== est) & (df['PERIODO']== per) & (df['COLE_CALENDARIO']== cal)
      if mask.sum()>100:
        p_val = round(test_sc.normaltest(df.loc[mask,'PUNT_GLOBAL']).pvalue,4)
        if p_val>=0.01: # Muestreme sólo los que no rechaza

          print(f'{p_val} PVALOR de la prueba de normalidad en el puntaje global para  estrato {est}, calendario {cal} y periodo {per}. #reg: {mask.sum()}')

0.2182 PVALOR de la prueba de normalidad en el puntaje global para  estrato Estrato 3, calendario A y periodo 20162. #reg: 700
0.0296 PVALOR de la prueba de normalidad en el puntaje global para  estrato Estrato 2, calendario A y periodo 20162. #reg: 1578
0.0188 PVALOR de la prueba de normalidad en el puntaje global para  estrato Estrato 4, calendario A y periodo 20162. #reg: 140
0.7438 PVALOR de la prueba de normalidad en el puntaje global para  estrato Estrato 4, calendario A y periodo 20152. #reg: 134
0.9435 PVALOR de la prueba de normalidad en el puntaje global para  estrato Estrato 4, calendario A y periodo 20142. #reg: 140
0.0264 PVALOR de la prueba de normalidad en el puntaje global para  estrato Estrato 5, calendario A y periodo 20194. #reg: 129
0.0719 PVALOR de la prueba de normalidad en el puntaje global para  estrato Sin Estrato, calendario A y periodo 20172. #reg: 114


In [68]:
mask = (df['FAMI_ESTRATOVIVIENDA']== 'Estrato 4') & (df['PERIODO']== 20142) & (df['COLE_CALENDARIO']== 'A')
px.histogram(df.loc[mask, 'PUNT_GLOBAL'], nbins=10, title=f'# {mask.sum()}')

In [69]:
mask = (df['FAMI_ESTRATOVIVIENDA']== 'Estrato 2') & (df['PERIODO']== 20162) & (df['COLE_CALENDARIO']== 'A')
px.histogram(df.loc[mask, 'PUNT_GLOBAL'], nbins=30, title=f'# {mask.sum()}')

In [70]:
mask = (df['FAMI_ESTRATOVIVIENDA']== 'Estrato 2') & (df['PERIODO']== 20162) & (df['COLE_CALENDARIO']== 'A')
df_analisis = df[mask].copy()
df_analisis

,PERIODO,COLE_CALENDARIO,FAMI_ESTRATOVIVIENDA,PUNT_GLOBAL,ANO
26,20162,A,Estrato 2,317.0,2016
47,20162,A,Estrato 2,231.0,2016
48,20162,A,Estrato 2,260.0,2016
75,20162,A,Estrato 2,348.0,2016
142,20162,A,Estrato 2,252.0,2016
...,...,...,...,...,...
49807,20162,A,Estrato 2,266.0,2016
49856,20162,A,Estrato 2,245.0,2016
49894,20162,A,Estrato 2,219.0,2016
49958,20162,A,Estrato 2,269.0,2016


Construcción de intervalo de confianza del 95% para la media del puntaje global en Periodo 20162	Calendario A	y Estrato 2

Testear la hipótesis de que la población de Periodo 20162	Calendario A	y Estrato 2 tiene una media de 250.

$H_0: \mu=250$ <br>
$H_1: \mu\neq250$ <br>

In [75]:
ttest(x=df_analisis['PUNT_GLOBAL'], y=250, confidence=0.95)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,12.408282,1577,two-sided,8.423680e-34,"[261.42, 265.71]",0.312362,1.978e+30,1.0


Con un nivel de confianza del 95% podemos decir que en el intervalo de 261.42 y 265.71 se encuentra la media de la población del   Periodo 20162 Calendario A y Estrato 2.

Se debe rechazar la hipótesis nula de que la media poblacional sea de 250.

# Use el ttest para comparar la media de dos poblaciones

$H_0$ $\mu_{P1}=\mu_{P2}$ <br>
$H_1$ $\mu_{P1}\neq \mu_{P2}$

In [76]:
mask_p1 = (df['FAMI_ESTRATOVIVIENDA']== 'Estrato 2') & (df['PERIODO']== 20162) & (df['COLE_CALENDARIO']== 'A')
df_p1 = df[mask_p1].copy()

mask_p2 = (df['FAMI_ESTRATOVIVIENDA']== 'Estrato 3') & (df['PERIODO']== 20162) & (df['COLE_CALENDARIO']== 'A')
df_p2 = df[mask_p2].copy()
ttest(x=df_p1['PUNT_GLOBAL'], y=df_p2['PUNT_GLOBAL'], confidence=0.95)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-10.895112,1334.966581,two-sided,1.533564e-26,"[-25.43, -17.67]",0.49554,4.725e+23,1.0


Nota para no rechazar la hipótesis nula el intervalo debe encerrar al 0.

** Se rechaza la hipótesis de igualdad y el intervalo de confianza me muestra que los de estrato 3 tienen una media más alta**

In [77]:
px.histogram(df_p1['PUNT_GLOBAL'])

In [78]:
px.histogram(df_p2['PUNT_GLOBAL'])

In [79]:
df_p2['PUNT_GLOBAL'].mean(), df_p1['PUNT_GLOBAL'].mean()

(285.1171428571429, 263.56717363751585)

El siguiente gráfico que los estratos 1 y 2 tienen un desempeño menor en las pruebas saber 11 a los estratos superiores. Además esos estratos abarcan gran porcentaje de las personas que presentan la prueba.

Se le pide que determine si la intervención se debe realizar para ambos estratos o sólo para uno de ellos.


In [80]:
px.box(df.sort_values('FAMI_ESTRATOVIVIENDA'), x='PUNT_GLOBAL', y='FAMI_ESTRATOVIVIENDA', orientation='h')

Nota: como las poblaciones de estrato 1 y 2 en general no se pueden asumir normales vamos a utilizar otro tipo de pruebas

Uso de la prueba kolmogorov-Smirnov para diferencias de distribuciones

$H_0$ : Las distribuciones son iguales.
$H_1$ : Las distribuciones son diferentes.

In [82]:

mask_p1 = (df['FAMI_ESTRATOVIVIENDA']== 'Estrato 1')
df_p1 = df[mask_p1].copy()

mask_p2 = (df['FAMI_ESTRATOVIVIENDA']== 'Estrato 2')
df_p2 = df[mask_p2].copy()
test_sc.ks_2samp(df_p1['PUNT_GLOBAL'], df_p2['PUNT_GLOBAL'])

KstestResult(statistic=0.16889799437730052, pvalue=1.2423950213700243e-124, statistic_location=239.0, statistic_sign=1)

# OJO con el tamaño y los dibujos

In [88]:
px.box(df.sample(100).sort_values('FAMI_ESTRATOVIVIENDA'), x='PUNT_GLOBAL', y='FAMI_ESTRATOVIVIENDA', orientation='h')